## Generate passwords

We use SHA1, SHA256, MD5 and bcrypt algorithms to hash generated passwords.

In [34]:
import random as rand
import string
from base64 import b64decode
import itertools
from Crypto.Protocol.KDF import bcrypt
from Crypto.Hash import MD5, SHA1, SHA256
from Crypto.Random import get_random_bytes
import pickle
from multiprocessing import Pool

rand.seed()

In [2]:
def read_from_file(name, size=None):
    with open(name) as f:
        data = f.read().split()
        if size:
            data = data[:size]
        return data


TOP_100 = read_from_file('top-100-passwords.txt', 100)
TOP_1M = read_from_file('top-1M-passwords.txt', 10**6)
TOP_10K = read_from_file('top-10k-passwords.txt', 10**4)
ENGLISH_COMMON_WORDS = read_from_file('english-common-words.txt')


def generate_random():
    size = rand.randint(5, 10)
    return ''.join(rand.choices(string.ascii_letters + string.digits + '!?', k=size))

def generate_from(pass_list):
    return rand.choice(pass_list)

def generate_random_readable():
    prepend_numbers = rand.random() > 0.5
    append_numbers = rand.random() > 0.5
    replace_symbols = rand.random() > 0.5
    
    num_words = rand.randint(2, 4)
    words = rand.sample(ENGLISH_COMMON_WORDS, num_words)
    result = []
    for word in words:
        if len(result) > 5:
            break
        result += list(word)
    
    if prepend_numbers:
        size = rand.randint(3, 5)
        result = rand.choices(string.digits, k=size) + result
    
    if append_numbers:
        size = rand.randint(2, 4)
        result += rand.choices(string.digits, k=size)
    
    replacements = {
        's': '$S5',
        'i': 'l1!',
        'a': '@A',
        't': '7T',
        'e': '3E',
        'g': '9G6',
        'o': 'O0',
        'b': '8B'
    }
    
    if replace_symbols:
        result = [rand.choice(replacements.get(el, '') + el) for el in result]
    
    return ''.join(result)
    
def generate_password():
    TOP_100_PERCENTAGE = 5
    TOP_1M_PERCENTAGE = 80
    RANDOM_PERCENTAGE = 5
    RANDOM_READABLE_PERCENTAGE = 10
    
    chance = rand.randint(1, 100)
    
    if chance <= TOP_100_PERCENTAGE:
        return generate_from(TOP_100)
    
    chance -= TOP_100_PERCENTAGE
    if chance <= TOP_1M_PERCENTAGE:
        return generate_from(TOP_1M)
    
    chance -= TOP_1M_PERCENTAGE
    if chance <= RANDOM_PERCENTAGE:
        return generate_random()
    
    chance -= RANDOM_PERCENTAGE
    assert chance <= RANDOM_READABLE_PERCENTAGE
    return generate_random_readable()

In [4]:
def generate_hash(algo, password):
    if type(password) is str:
        password = password.encode()
    return algo.new(password).hexdigest()

def generate_bcrypt_hash(password, cost):
    salt = get_random_bytes(16)
    return bcrypt(password.encode(), cost, salt).decode(), salt.hex()

def generate_hashes():
    NUM_PASSWORDS = 500_000
    
    for algo in MD5, SHA1, SHA256:
        name = algo.__name__.split('.')[-1]
        with open(f'part1/{name}.csv', 'w') as f:
            for i in range(NUM_PASSWORDS):
                print(generate_hash(algo, generate_password()), file=f)
    
    BCRYPT_COST = 4
    with open(f'part1/bcrypt-{BCRYPT_COST}.csv', 'w') as f:
        for i in range(NUM_PASSWORDS):
            print(*generate_bcrypt_hash(generate_password(), BCRYPT_COST), sep=',', file=f)


In [42]:
generate_hashes()

## Find hash preimages

In [19]:
def count_restored_passwords(args):
    passwd_list, hashed, algo = args
    
    assert type(hashed) is set
    algos = {'md5': MD5, 'sha1': SHA1}
    algo = algos[algo]
    return [el for el in passwd_list if generate_hash(algo, el) in hashed]


def batches(it, size):
    it = iter(it)
    done = False
    while not done:
        result = []
        try:
            for i in range(size):
                result.append(next(it))
        except StopIteration:
            done = True
        if result:
            yield result

def parallel_restore(paswds, hashes, algo, batch_size):
    with Pool() as pool:
        args = ((batch, hashes, algo) for batch in batches(paswds, batch_size))
        return sum(pool.imap_unordered(count_restored_passwords, args), [])


def restore_passwords_simple():
    print('Restore passwords using a dictionary of passwords (brute-force)')
    print()
    
    print('MD5 (dictionary - 1M)')
    with open('part2/md5.csv') as f:
        f.readline() # ignore the header
        hashes = {b64decode(el).hex() for el in f.read().split()}
    
    md5_restored = %time parallel_restore(TOP_1M, hashes, 'md5', 10_000)
    print(f'Restored {len(md5_restored)} passwords ({len(md5_restored) / len(hashes) * 100:.2f}%)')
    print('First 10 passwords', md5_restored[:10])
    print()
    
    print('SHA1 with salt (dictionary - 100)')
    with open('part2/sha1.csv') as f:
        f.readline() # ignore the header
        hashes, salts = set(), []
        for row in f.read().split():
            h, s = row.split(',')
            hashes.add(b64decode(h).hex())
            salts.append(s)
    
    SALT_LEN = len(salts[0])
    passwd_salt = map(''.join, itertools.product(TOP_100, salts))
    sha1_restored = %time parallel_restore(passwd_salt, hashes, 'sha1', 10_000)
    sha1_restored = list({el[:len(el) - SALT_LEN] for el in sha1_restored})
    
    print(f'Restored {len(sha1_restored)} passwords ({len(sha1_restored) / len(hashes) * 100:.2f}%)')
    print('First 10 passwords', sha1_restored[:10])
    
    return md5_restored, sha1_restored
    

In [20]:
md5_restored, sha1_restored = restore_passwords_simple()

Restore passwords using a dictionary of passwords (brute-force)

MD5 (dictionary - 1M)
CPU times: user 1.19 s, sys: 102 ms, total: 1.29 s
Wall time: 7.6 s
Restored 7926 passwords (32.44%)
First 10 passwords ['123456', 'password', '12345678', 'qwerty', '123456789', '12345', '1234', '111111', '1234567', 'dragon']

SHA1 with salt (dictionary - 10K)
CPU times: user 47.1 s, sys: 10.5 s, total: 57.6 s
Wall time: 1min 41s
Restored 62 passwords (0.06%)
First 10 passwords ['master', 'shadow', 'harley', 'matrix', 'dragon', 'jordan', '121212', '123456789', 'ashley', 'charlie']


In [25]:
"""Rainbow table implementation.
This is just an example implementation and it doesn't really recover any passwords from the
hashes in the part2/ directory. The reason is that the table is just too small to do anything useful.
"""

def get_gen_fn(plaintext_size, letter_set):
    return lambda: ''.join(rand.choices(letter_set, k=plaintext_size))

def hash_md5(s):
    return MD5.new(s.encode()).hexdigest()

def get_reduce_fn(plaintext_size, letter_set):
    def inner(hashed, i):
        val = (int(hashed, 16) ^ i) % len(letter_set)**plaintext_size
        result = ''
        
        for _ in range(plaintext_size):
            result += letter_set[val % len(letter_set)]
            val //= len(letter_set)
        
        return result
    
    return inner

class RainbowTable:
    def __init__(self, *, chain_len, hash_fn, reduce_fn, gen_fn):
        self.chain_len = chain_len
        self.hash_fn = hash_fn
        self.reduce_fn = reduce_fn
        self.gen_fn = gen_fn
    
    def save(self, f):
        pickle.dump(self.table, f)
    
    def load(self, f):
        self.table = pickle.load(f)
    
    def generate(self, size):
        self.table = {}
        
        for _ in range(size):
            start = self.gen_fn()
            curr = start
            for i in range(self.chain_len):
                hashed = self.hash_fn(curr)
                curr = self.reduce_fn(hashed, i)
            self.table[hashed] = start
    
    def restore(self, hashed):
        for start_col in range(self.chain_len - 1, -1, -1):
            curr = hashed
            for i in range(start_col, self.chain_len - 1):
                curr = self.hash_fn(self.reduce_fn(curr, i))
            
            if curr in self.table:
                maybe_result = self._traverse_chain(self.table[curr], hashed)
                if maybe_result:
                    return maybe_result
        
        return None
    
    def _traverse_chain(self, start, hashed):
        for i in range(self.chain_len):
            next_hash = self.hash_fn(start)
            if next_hash == hashed:
                return start
            start = self.reduce_fn(next_hash, i)
        
        return None
    